# load

> Load documents from GCS

In [1]:
#| default_exp load

In [2]:
#| export
from typing import Dict, Any, Iterable, List, Tuple
import pandas as pd

from pydantic import BaseModel
from sklearn.model_selection import train_test_split

from classifier.schema import PROJECT_BUCKET, WRITE_PREFIX

Load the data from GCS

In [3]:
#| export
RAW_EMAILS_FILE = "Last50KCases_withSubjectAndBody.xlsx"
TEJAS_FILE = "train_test_split/pd_3k_cases_cleaned.csv"

In [4]:
training_data_sample = pd.read_excel(
    f"gs://{PROJECT_BUCKET}/Last50KCases_withSubjectAndBody.xlsx", nrows=10)
training_data_sample.loc[:, 'email_subject'] = training_data_sample.email_subject.fillna("N/A")
training_data_sample.head()

,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_category,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,email_subject,email_body
0,SPD,3469839,NaN,2023-09-11T13:22:32,Order Processing,Order Entry,Order Processing,NaN,2,0.876806,Correct,8,PO# 7004014842 || Walgreens Store 16422 || Ohi...,External Email â€“ Please use caution before o...
1,PD,3469841,a1G4z00000H4wVrEAJ,2023-09-11T13:22:37,Order Processing,Drop Ship Order,Order Processing,Drop Ship Order,1,0.838383,Correct,8,RE: Drop Ship Invoices,"Hi, Tara. Thank you for your patience. I have..."
2,PD,3469842,a1G4z00000H4uvREAR,2023-09-11T13:22:43,Order Processing,Order Entry,General Inquiry,NaN,1,0.838036,Incorrect,8,Purchase Order #65398,External Email â€“ Please use caution before o...
3,PD,3469844,a1G4z00000H4vF7EAJ,2023-09-11T13:22:47,Order Processing,Order Entry,General Inquiry,NaN,1,0.838036,Incorrect,8,Purchase Order #SSC2320930,External Email â€“ Please use caution before o...
4,SPD,3469845,NaN,2023-09-11T13:23:01,Order Processing,Order Entry,Order Processing,NaN,2,0.794613,Correct,7,Krystexxa PIT 09.11.2023,External Email â€“ Please use caution before o...


In [5]:
#| export
def get_raw_emails(**read_excel_kwargs) -> pd.DataFrame:
    return pd.read_excel(f'gs://{PROJECT_BUCKET}/{RAW_EMAILS_FILE}', **read_excel_kwargs)

In [6]:
#| export
LABEL_COLUMN = "sfdc_category"

def process_raw_emails(emails: pd.DataFrame) -> pd.DataFrame:
    emails = emails.copy().rename({LABEL_COLUMN: 'label'}, axis=1)
    emails.loc[:, 'email_subject'] = emails.email_subject.fillna("N/A").astype(str)
    emails.loc[:, 'email_body'] = emails.email_body.fillna("N/A").astype(str)
    return emails

In [7]:
training_data_sample_processed = process_raw_emails(training_data_sample)
training_data_sample_processed.label.value_counts()

label
Order Processing    9
Product Inquiry     1
Name: count, dtype: int64

In [8]:
#| export
def get_possible_labels() -> List[str]:
    labels = get_raw_emails(usecols=[LABEL_COLUMN])
    return labels[LABEL_COLUMN].unique().tolist()

In [9]:
possible_labels = get_possible_labels()
possible_labels

['Order Processing',
 'Product Inquiry',
 'Account/Inquiry',
 'General Inquiry',
 'Returns',
 'Billing / Invoice',
 'Delivery',
 'Credits',
 'Order Discrepancy',
 'Pricing',
 'Program / Promotions']

Get Tejas index

In [10]:
tejas_file_sample = pd.read_csv(f'gs://{PROJECT_BUCKET}/{TEJAS_FILE}', nrows=10)
tejas_file_sample.head(2)

,case_number,email_subject,clean_email_body,sfdc_category,baseline_category,category
0,3607450,Sodium Thiosulfate dropship order,Account 2057190482 Please place an order for ...,Order Processing,Order Processing,Order Processing
1,3604094,Credit Rebill Request,"Good afternoon, I would like to request a cre...",Billing / Invoice,Credits,Billing / Invoice


In [11]:
#| export
def get_tejas_case_numbers() -> pd.Series:
    return pd.read_csv(f'gs://{PROJECT_BUCKET}/{TEJAS_FILE}', usecols=['case_number']).case_number

In [12]:
tejas_case_numbers = get_tejas_case_numbers()
# Are these case numbers unique?
assert tejas_file_sample.duplicated().sum() == 0

In [13]:
training_data_sample.columns

Index(['BU', 'case_number', 'ACCOUNT_BUSINESS_UNIT__C', 'received_at',
       'sfdc_category', 'sfdc_subcategory', 'predicted_category',
       'predicted_subcategory', 'record_type', 'probability', 'Accuracy_upd',
       'Bin', 'email_subject', 'email_body'],
      dtype='object')

In [14]:
training_data_sample.merge(
    tejas_case_numbers, how='right', on='case_number').dropna()

,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_category,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,email_subject,email_body


In [15]:
#| export
def get_raw_emails_tejas_case_numbers() -> pd.DataFrame:
    raw_emails = get_raw_emails()
    tejas_case_numbers = get_tejas_case_numbers().tolist()
    raw_emails = raw_emails[raw_emails.case_number.isin(tejas_case_numbers)]
    return raw_emails

In [16]:
raw_emails_tejas = get_raw_emails_tejas_case_numbers()

In [17]:
raw_emails_tejas.head(2)

,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_category,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,email_subject,email_body
19,PD,3469874,NaN,2023-09-11T13:30:10,Account/Inquiry,Account updates,Account/Inquiry,NaN,1,0.479256,Correct,4,Requesting Weblink login - 2 Cedra accounts,"Good morning, Please send Weblink username em..."
29,PD,3469907,a1G4z00000FwveoEAB,2023-09-11T13:40:44,Delivery,Customer Request,Billing / Invoice,NaN,1,0.406604,Incorrect,4,Offboarding for IONE PHARMACY 340B,"Hello Team, Kindly assist on submitting a req..."


## Define our TrainingInstance

In [18]:
#| export
class Email(BaseModel):
    idx: int
    label: str
    email_subject: str
    email_body: str
    metadata: Dict[str, Any]

    def to_series(self) -> pd.Series:
        data = self.metadata.copy()
        data['idx'] = self.idx
        data['label'] = self.label
        data['email_subject'] = self.email_subject
        data['email_body'] = self.email_body
        return pd.Series(data)        


def email_from_row(
        idx: int, 
        row: pd.Series,
        label_column: str = LABEL_COLUMN):
    metadata = row.drop(
        [
            label_column, 
            'email_subject',
            'email_body'
        ]).to_dict()
    return Email(
        idx=idx,
        label=row[label_column],
        email_subject=str(row.email_subject),
        email_body=str(row.email_body),
        metadata=metadata
    )

In [20]:
example_emails = email_from_row(0, training_data_sample.iloc[0])
example_emails

Email(idx=0, label='Order Processing', email_subject='PO# 7004014842 || Walgreens Store 16422 || Ohio State University', email_body='External Email â€“ Please use caution before opening attachments or clicking links  Cardinal Ordering Team,  Please place the drop ship order(s) listed below for:  Client Name Ohio State University PO ID 7004014842 Account # 2150126632 Store # 16422 NDC 70127010010 Drug Name EPIDIOLEX 100MG/ML SOL 100ML Order Quantity 5 Prescriber Name LUCRETIA LONG, PHILIP CLAYTON JONAS Prescriber NPI or DEA ML0822634, FJ1422132  Thanks & Regards, Bhavesh Lalwani', metadata={'BU': 'SPD', 'case_number': 3469839, 'ACCOUNT_BUSINESS_UNIT__C': nan, 'received_at': '2023-09-11T13:22:32', 'sfdc_subcategory': 'Order Entry', 'predicted_category': 'Order Processing', 'predicted_subcategory': nan, 'record_type': 2, 'probability': 0.8768061, 'Accuracy_upd': 'Correct', 'Bin': 8})

## Get a "training" and "test" sample, remove outliers

### Outlier removal
We will remove outliers by subject + body length

In [21]:
#| export
# Our prompt to summarize takes up some amount of prompt space. This is a rough limit
EMAIL_SIZE_LIMIT = 7800


def email_small_enough(subject: str, body: str, limit: int = EMAIL_SIZE_LIMIT) -> bool:
    if not isinstance(subject, str):
        subject = str(subject)
    if not isinstance(body, str):
        body = str(body)
    return (len(subject) + len(body)) < limit

In [22]:
size_mask = raw_emails_tejas.apply(
    lambda row: email_small_enough(
        row.email_subject,
        row.email_body
    ), axis=1)

In [23]:
training_data_included = raw_emails_tejas[size_mask]
training_data_included.shape[0]

3137

In [24]:
training_data_included.head(2)

,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_category,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,email_subject,email_body
19,PD,3469874,NaN,2023-09-11T13:30:10,Account/Inquiry,Account updates,Account/Inquiry,NaN,1,0.479256,Correct,4,Requesting Weblink login - 2 Cedra accounts,"Good morning, Please send Weblink username em..."
29,PD,3469907,a1G4z00000FwveoEAB,2023-09-11T13:40:44,Delivery,Customer Request,Billing / Invoice,NaN,1,0.406604,Incorrect,4,Offboarding for IONE PHARMACY 340B,"Hello Team, Kindly assist on submitting a req..."


### Train and test set
We will sample by label. Limit to 5,000 emails total. 90% train, 10% test.

In [25]:
#| export
INCLUSION_COUNT = 3000


def get_train_test_idx(
        data: pd.DataFrame,
        inclusion_count: int = INCLUSION_COUNT, 
        train_proportion: int = 0.8,
        label_column: str = LABEL_COLUMN,
        random_state: int = 42):
    train_count = int(round(inclusion_count * train_proportion))
    test_count = inclusion_count - train_count
    train, test = train_test_split(
        data,
        test_size=test_count, 
        train_size=train_count,
        random_state=random_state,
        stratify=data[label_column])
    input_data = pd.concat([train, test], axis=0)
    return train_test_split(
        input_data,
        test_size=1-train_proportion,
        train_size=train_proportion,
        random_state=random_state,
        stratify=input_data[label_column]
    )

In [26]:
train, test = get_train_test_idx(training_data_included)

In [27]:
train.shape[0] + test.shape[0]

3000

In [28]:
train[LABEL_COLUMN].value_counts().sort_index(), test[LABEL_COLUMN].value_counts().sort_index()

(sfdc_category
 Account/Inquiry         658
 Billing / Invoice       126
 Credits                  47
 Delivery                 64
 General Inquiry          35
 Order Discrepancy       215
 Order Processing        898
 Pricing                  10
 Product Inquiry         131
 Program / Promotions     13
 Returns                 203
 Name: count, dtype: int64,
 sfdc_category
 Account/Inquiry         164
 Billing / Invoice        31
 Credits                  12
 Delivery                 16
 General Inquiry           9
 Order Discrepancy        54
 Order Processing        224
 Pricing                   3
 Product Inquiry          33
 Program / Promotions      3
 Returns                  51
 Name: count, dtype: int64)

In [29]:
#| export
TRAIN_IDX_NAME = "train_idx.csv"
TEST_IDX_NAME = "test_idx.csv"


def write_idx(
        train_idx: pd.Index, 
        test_idx: pd.Index, 
        bucket_name: str = PROJECT_BUCKET,
        prefix: str = WRITE_PREFIX):
    
    train_idx.to_series().to_csv(f"gs://{bucket_name}/{prefix}/{TRAIN_IDX_NAME}", index=False)
    test_idx.to_series().to_csv(f"gs://{bucket_name}/{prefix}/{TEST_IDX_NAME}", index=False)

In [30]:
write_idx(
    train.index,
    test.index,
    prefix=f"{WRITE_PREFIX}/tejas"
)

In [31]:
#| export
def get_idx(
        bucket_name: str = PROJECT_BUCKET,
        prefix: str = WRITE_PREFIX) -> Tuple[pd.Series, pd.Series]:
    return pd.read_csv(f'gs://{bucket_name}/{prefix}/{TRAIN_IDX_NAME}').iloc[:, 0], \
        pd.read_csv(f'gs://{bucket_name}/{prefix}/{TEST_IDX_NAME}').iloc[:, 0]

In [32]:
train_idx, test_idx = get_idx(prefix=f"{WRITE_PREFIX}/tejas")
train_idx.head()

0    31716
1    35200
2      462
3     3705
4    25300
Name: 0, dtype: int64

In [33]:
full_idx = pd.concat(
    [train_idx, test_idx],
    axis=0,
    ignore_index=True
)
training_data_included.loc[full_idx, :].head(2)

,BU,case_number,ACCOUNT_BUSINESS_UNIT__C,received_at,sfdc_category,sfdc_subcategory,predicted_category,predicted_subcategory,record_type,probability,Accuracy_upd,Bin,email_subject,email_body
31716,PD,3598350,NaN,2023-11-01T19:40:57,Order Processing,Drop Ship Order,Order Processing,Drop Ship Order,1,0.576672,Correct,5,Equashield latest - FW: EQ II Catalog 2023 - C...,External Email â€“ Please use caution before o...
35200,PD,3613116,NaN,2023-11-08T17:27:04,Billing / Invoice,Account balance,Billing / Invoice,NaN,1,0.496874,Correct,4,Auto-Reply. We Have Received Your Request,"To whom it may concern, Your request has been..."


In [34]:
#| export
def get_emails_from_frame(
        data: pd.DataFrame,
        which: str = 'both',
        bucket_name: str = PROJECT_BUCKET,
        index_prefix: str = WRITE_PREFIX,
        label_column: str = LABEL_COLUMN
) -> Iterable[Email]:
    """
    Pass a raw dataframe
    """
    if which not in ['train', 'test', 'both']:
        raise ValueError("which must be one of 'train', 'test', 'both'")
    # Load train and test idx
    train_idx, test_idx = get_idx(bucket_name=bucket_name, prefix=index_prefix)
    full_idx = pd.concat([train_idx, test_idx], axis=0, ignore_index=True)
    if which == 'train':
        data = data.loc[train_idx, :]
    elif which == 'test':
        data = data.loc[test_idx, :]
    else:  # Both
        data = data.loc[full_idx, :]
    for idx, row in data.iterrows():
        yield email_from_row(idx, row, label_column=label_column)

In [35]:
# max length
training_instance_loader = get_emails_from_frame(
    training_data_included, 
    'train',
    index_prefix=f"{WRITE_PREFIX}/tejas")
test_instance_loader = get_emails_from_frame(
    training_data_included, 
    'test',
    index_prefix=f"{WRITE_PREFIX}/tejas")

In [36]:
#| export
def get_batches(loader: Iterable[Any], batch_size: int = 32) -> Iterable[List[Any]]:
    "Get a batch of anything from an iterable."
    batch = []
    for item in loader:
        batch.append(item)
        if len(batch) >= batch_size:
            yield batch
            batch = []
    yield batch

In [37]:
next(training_instance_loader)

Email(idx=31716, label='Order Processing', email_subject='Equashield latest - FW: EQ II Catalog 2023 - Cardinal Health.xlsx', email_body='External Email â€“ Please use caution before opening attachments or clicking links  Let us know if you need anything else.    Regards,  Thomas Everitt Customer Service Representative Office    +1 516 684 8200 / Ext: 220 Mobile  +1 516 398 97 25 Fax          +1 516 684 8202 www.equashield.com<http://www.equashield.com/> [cid:image001.png@01DA0CD9.861376B0] [cid:image002.png@01DA0CD9.861376B0]  From: Pavlina Georgieva <pavlina@equashield.com> Sent: Wednesday, November 1, 2023 3:39 PM To: Thomas Everitt <Thomas.e@equashield.com> Subject: EQ II Catalog 2023 - Cardinal Health.xlsx      Regards,  Pavlina Georgieva Logistics Coordinator Office    +1 516 684 8200 / Ext: 202 Fax          +1 516 684 8202 www.equashield.com<http://www.equashield.com/> [cid:image001.png@01DA0CD9.861376B0] [cid:image002.png@01DA0CD9.861376B0]   ________________________________ Co

In [38]:
next(test_instance_loader)

Email(idx=13614, label='Returns', email_subject='Need signature AC account', email_body='Good afternoon,  We have received an order from customer 2057194105. They sent unsigned MRA 3901356789. Can you please reach out to the customer and let them know they have 48 hours to send a signed MRA or we will send back the case for no credit.  Thanks,   [cid:image001.png@01D9F52F.5AD273F0]  Tom Coppedge Returns Lead | Warehouse Operations 2840 Elm Pont Industrial Drive St. Charles, MO. 63301    _________________________________________________  This message is for the designated recipient only and may contain privileged, proprietary or otherwise private information. If you have received it in error, please notify the sender immediately and delete the original. Any other use of the email by you is prohibited.  Dansk - Deutsch - Espanol - Francais - Italiano - Japanese - Nederlands - Norsk - Portuguese - Chinese Svenska: http://www.cardinalhealth.com/en/support/terms-and-conditions-english.html'

## Export

In [39]:
#| hide
import nbdev; nbdev.nbdev_export()